In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train=pd.read_csv("../input/leaf-classification/train.csv.zip",index_col='id')

df_train.shape

In [ ]:
df_train.isna().sum()

In [ ]:
y_train=df_train.species
X_train=df_train.drop(columns=['species'],axis=1)

In [ ]:
object_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and X_train[cname].dtype == "object"]
object_cols

In [ ]:
len(y_train.value_counts())

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_fit = encoder.fit(y_train)
y_train = y_fit.transform(y_train)
classes = list(y_fit.classes_) 

In [ ]:
X_train.head()

In [ ]:
from sklearn.preprocessing import QuantileTransformer
quantile_transformer = QuantileTransformer(random_state=0)
scaler = quantile_transformer.fit(X_train)
X_train= quantile_transformer.transform(X_train)

In [ ]:
parameters = {
    'tol':[0.001, 0.009],
    'C':list(range(100, 1000,100)),
    'max_iter': list(range(10, 100, 10)),
    "solver":("newton-cg", "lbfgs", "liblinear"),
    "penalty":("l1", "l2")
}
parameters

In [ ]:
my_randome_state=500

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model  import LogisticRegression
from sklearn.metrics import log_loss

log_reg =LogisticRegression(multi_class='multinomial',
                            random_state=my_randome_state)

gsearch = GridSearchCV(estimator=log_reg,
                       param_grid = parameters,
                       scoring="neg_log_loss",
                       n_jobs=4,cv=5, verbose=7)

In [ ]:
gsearch.fit(X_train, y_train)

In [ ]:
best_max_iter = gsearch.best_params_.get('max_iter')
best_tol = gsearch.best_params_.get('tol')
best_C = gsearch.best_params_.get('C')
best_solver = gsearch.best_params_.get('solver')
best_penalty = gsearch.best_params_.get('penalty')
best_max_iter,best_tol,best_C,best_penalty,best_solver

In [ ]:
final_model = LogisticRegression(max_iter=best_max_iter,
                                 random_state=my_randome_state, 
                                 tol=best_tol,
                                 C=best_C,
                                 solver=best_solver,
                                 penalty=best_penalty)

In [ ]:
final_model.fit(X_train, y_train)

# test

In [ ]:
X_test=pd.read_csv("../input/leaf-classification/test.csv.zip")

In [ ]:
test_ids = X_test.id
X_test = X_test.drop(['id'], axis =1)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
final_predictions = final_model.predict_proba(X_test)

In [ ]:
submission = pd.DataFrame(final_predictions, columns=classes)
submission.insert(0, 'id', test_ids)
submission

In [ ]:
submission.to_csv('submission_log_reg.csv', index = False)
print("done")